In [2]:
import pandas as pd
import json

with open('item_nameid.json', 'r') as f:
    data = json.load(f)

df = pd.DataFrame(list(data.items()), columns=['item_name', 'name_id'])

In [3]:
df

,item_name,name_id
0,'Blueberries' Buckshot | NSWC SEAL,176209284
1,'Medium Rare' Crasswater | Guerrilla Warfare,176262691
2,'The Doctor' Romanov | Sabre,176096416
3,'Two Times' McCoy | TACP Cavalry,176209278
4,'Two Times' McCoy | USAF TACP,176096507
...,...,...
26563,StatTrak™ MAG-7 | MAGnitude (Factory New),176550793
26564,★ Skeleton Knife | Ultraviolet (Factory New),176507417
26565,★ Survival Knife | Marble Fade (Minimal Wear),176507246
26566,FAMAS | Yeti Camo (Field-Tested),176504424


In [6]:
import requests
import re


def get_orderbook(nameid):
    url = f'https://steamcommunity.com/market/itemordershistogram?country=US&language=english&currency=1&item_nameid={nameid}'
    response = requests.get(url)
    data = response.json()
    bids = {}
    asks = {}

    # ---- BIDS (descending prices) ----
    buy_graph = data["buy_order_graph"]

    prev_volume = 0
    for price, cumulative_volume, _ in buy_graph:
        level_volume = cumulative_volume - prev_volume
        if level_volume > 0:
            bids[price] = level_volume
        prev_volume = cumulative_volume

    # ---- ASKS (ascending prices) ----
    sell_graph = data["sell_order_graph"]

    prev_volume = 0
    for price, cumulative_volume, _ in sell_graph:
        level_volume = cumulative_volume - prev_volume
        if level_volume > 0:
            asks[price] = level_volume
        prev_volume = cumulative_volume



get_orderbook(176550793)

({1.9: 4,
  1.89: 2,
  1.88: 99,
  1.78: 2,
  1.72: 4,
  1.7: 25,
  1.69: 1,
  1.68: 30,
  1.65: 3,
  1.63: 15,
  1.62: 99,
  1.61: 1,
  1.56: 1,
  1.55: 20,
  1.53: 1,
  1.52: 1,
  1.41: 1,
  1.3: 1,
  1.28: 1,
  1.21: 28,
  1.11: 45,
  0.99: 11,
  0.73: 40,
  0.58: 40,
  0.57: 50,
  0.47: 50,
  0.34: 35,
  0.32: 10,
  0.28: 200,
  0.23: 80,
  0.13: 2800,
  0.06: 2020,
  0.05: 1,
  0.04: 2500,
  0.03: 1},
 {2.1: 1,
  2.12: 1,
  2.15: 1,
  2.2: 2,
  2.21: 3,
  2.26: 1,
  2.31: 1,
  2.36: 1,
  2.46: 1,
  2.5: 1,
  2.55: 1,
  2.59: 2,
  2.65: 1,
  2.69: 1,
  2.7: 1,
  2.73: 1,
  2.77: 1,
  2.81: 2,
  2.82: 1,
  2.83: 1,
  2.88: 1,
  2.94: 1,
  3: 3,
  3.05: 1,
  3.06: 1,
  3.1: 3,
  3.13: 1,
  3.15: 1,
  3.16: 1,
  3.19: 1,
  3.26: 1,
  3.3: 1,
  3.31: 1,
  3.38: 1,
  3.4: 1,
  3.41: 1,
  3.5: 1,
  3.6: 1,
  3.65: 1,
  3.74: 1,
  3.77: 2,
  3.81: 1,
  3.88: 1,
  3.94: 1,
  3.95: 1,
  3.96: 1})

In [ ]:
import requests
from lxml import html
from robust_json import RobustJSONParser
import json
from jsonpath_ng import jsonpath, parse

def scrape_steam_market(url):
    """
    Scrape Steam market page using requests instead of Scrapy.
    This works better in Jupyter notebooks.
    """
    # Add headers to mimic a browser
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        raise Exception(f"Failed to fetch page: {response.status_code}")
    
    # Parse HTML
    tree = html.fromstring(response.content)
    
    # Extract script containing market data
    scripts = tree.xpath("//script[contains(., 'g_rgAppContextData')]/text()")

    # Find all listing divs
    listing_divs = tree.xpath('//div[starts-with(@id, "listing_")]')
    for div in listing_divs:
        print(html.tostring(div, pretty_print=True, encoding='unicode'))
    
    if scripts:
        
        parser = RobustJSONParser()
        jsonpath_expr = parse('$[0]["730"]["2"].*')
        res = parser.parse_all(scripts[0])
        matches = jsonpath_expr.find(res)
        for match in matches:
            print(json.dumps(match.value, indent=4))
        return res

    else:
        return None

In [ ]:
# Example usage: Scrape a specific item
nova_url = "https://steamcommunity.com/market/listings/730/Nova%20%7C%20Hyper%20Beast%20(Factory%20New)"
ak_url = scrape_steam_market("https://steamcommunity.com/market/listings/730/AK-47%20%7C%20Redline%20%28Field-Tested%29")
#nova = scrape_steam_market(nova_url)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
response = requests.get(nova_url, headers=headers)

if response.status_code != 200:
    raise Exception(f"Failed to fetch page: {response.status_code}")







<div class="market_listing_row market_recent_listing_row listing_778696029365402121" id="listing_778696029365402121">
	
	<div class="market_listing_item_img_container">		<img id="listing_778696029365402121_image" src="https://community.fastly.steamstatic.com/economy/image/i0CoZ81Ui0m-9KwlBY1L_18myuGuq1wfhWSaZgMttyVfPaERSR0Wqmu7LAocGIGz3UqlXOLrxM-vMGmW8VNxu5Dx60noTyLwlcK3wiFO0POlPPNSI_-RHGavzOtyufRkASq2lkxx4W-HnNyqJC3FZwYoC5p0Q7FfthW6wdWxPu-371Pdit5HnyXgznQeHYY5wyA/62fx62f" srcset="https://community.fastly.steamstatic.com/economy/image/i0CoZ81Ui0m-9KwlBY1L_18myuGuq1wfhWSaZgMttyVfPaERSR0Wqmu7LAocGIGz3UqlXOLrxM-vMGmW8VNxu5Dx60noTyLwlcK3wiFO0POlPPNSI_-RHGavzOtyufRkASq2lkxx4W-HnNyqJC3FZwYoC5p0Q7FfthW6wdWxPu-371Pdit5HnyXgznQeHYY5wyA/62fx62f 1x, https://community.fastly.steamstatic.com/economy/image/i0CoZ81Ui0m-9KwlBY1L_18myuGuq1wfhWSaZgMttyVfPaERSR0Wqmu7LAocGIGz3UqlXOLrxM-vMGmW8VNxu5Dx60noTyLwlcK3wiFO0POlPPNSI_-RHGavzOtyufRkASq2lkxx4W-HnNyqJC3FZwYoC5p0Q7FfthW6wdWxPu-371Pdit5HnyXgznQeHYY5wyA/

<Element html at 0x7f0c6a38d130>

In [ ]:
# Parse HTML
tree = html.fromstring(response.content)

script = tree.xpath("//script[contains(., 'g_rgAppContextData')]/text()")[0]
parser = RobustJSONParser()
res = parser.parse_all(script)

pd.DataFrame(columns=['listing_id', 'price', 'float_value', 'int_value',])

listing_ids = list(res[1].keys())
f = 1
for id in listing_ids:
    print(id)
    print(f"price: {res[1][id]['price'] / 100:.2f} USD")
    assetid = res[1][id]['asset']["id"]
    asset_properties = res[0]["730"]["2"][assetid]["asset_properties"]
    for x in asset_properties:
        prop = list(x.keys())[1]
        print(f"{prop}: {x[prop]}")




787702594481086323
price: 47.83 USD
int_value: 695
float_value: 0.0602021068334579468
string_value: F6E6064072134EF7EED5D66FF2DEF3C6F2CE0A5A2C1DF5B641F39E7C767676FA86FE34F72CB6
785450135140273825
price: 48.55 USD
int_value: 323
float_value: 0.0419858098030090332
string_value: D3C34F142A446BD2CBF0F34AD7FBD6E3D7EB03217C3AD09310D1B1D6DBD1C370FDBB59535353DFA3D76B6096A6
780946204105716462
price: 400.00 USD
int_value: 655
float_value: 0.0680974274873733521
string_value: 5F4FA198B1B265477C7FC65B775A6F5B67F5B2F2B35C1FD05A37742F5B05767045
748286671361888433
price: 60.00 USD
int_value: 950
float_value: 0.0658750906586647034
string_value: 4252D382F8F5635A6162DB466A4772467AC791D9AE4102F4452A5A324A5BBEA1FE
758427294157815170
price: 60.60 USD
int_value: 577
float_value: 0.0491664521396160126
string_value: 2131F7A2C8BD17390201B8250924112519E2E484CB2261E02549225129BCA3047A
756170421080074575
price: 60.89 USD
int_value: 785
float_value: 0.0602238476276397705
string_value: 4F5FFEB089C06E576C6FD64B674A7F

In [134]:
res = requests.get("https://steamcommunity.com/market/listings/730/Nova%20|%20Hyper%20Beast%20(Factory%20New)/render/?query=&start=0&count=10&country=US&language=english&currency=1")

In [135]:
data = json.loads(res.content)
listings = list(data["listinginfo"].keys())
assets = data["assets"]["730"]["2"]
for listing in listings:
    row = {}
    row["id"] = listing
    row["price"] = (data["listinginfo"][listing]["converted_price"] + data["listinginfo"][listing]["converted_fee"]) / 100
    row["asset_id"] = data["listinginfo"][listing]["asset"]["id"]
    for prop in assets[row["asset_id"]]["asset_properties"]:
        if prop['propertyid'] == 2:
            row["float_value"] = float(prop['float_value'])
            break
    

    print(row)

{'id': '787702594481086323', 'price': 64.81, 'asset_id': '49604008816', 'float_value': 0.06020210683345795}
{'id': '785450135140273825', 'price': 65.8, 'asset_id': '49442349980', 'float_value': 0.04198580980300903}
{'id': '780946204105716462', 'price': 66.24, 'asset_id': '15799657470', 'float_value': 0.06809742748737335}
{'id': '748286671361888433', 'price': 69.0, 'asset_id': '8974671889', 'float_value': 0.0658750906586647}
{'id': '758427294157815170', 'price': 69.69, 'asset_id': '14555955670', 'float_value': 0.04916645213961601}
{'id': '756170421080074575', 'price': 70.01, 'asset_id': '8890990513', 'float_value': 0.06022384762763977}
{'id': '797834702520144439', 'price': 70.73, 'asset_id': '10570948232', 'float_value': 0.05555233731865883}
{'id': '645829214576069346', 'price': 75.0, 'asset_id': '16890732704', 'float_value': 0.04942776635289192}
{'id': '774182369726066308', 'price': 75.0, 'asset_id': '47311398843', 'float_value': 0.06913784891366959}
{'id': '732528425942744347', 'price

In [138]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
from playwright.async_api import async_playwright
import json

TARGET_URL = "https://steamcommunity.com/market/listings/730/Nova%20%7C%20Hyper%20Beast%20(Factory%20New)"

async def run():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        async def handle_response(response):
            if "render" in response.url and response.request.resource_type == "xhr":
                try:
                    data = await response.json()
                    print("Captured JSON:")
                    print(json.dumps(data, indent=2)[:1000])
                except:
                    pass

        page.on("response", handle_response)

        await page.goto(TARGET_URL)
        await page.wait_for_timeout(5000)

        await browser.close()

await run()